In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.optimize as sopt

# Generate Random Data
The following block generates random test data. Later, I will write code that converts the true data to the same format. Note that with the following method, the true solution when m goes towards infinity is $\alpha$ = 0. However, since the randomly generated data is imperfect, the computed solution will be something else. If $m$ is large enough, then the magnitude of the $\alpha$s should be approximately the same.

In [ ]:
def generate_data(m, n, r):
    #todo: add print messages for profiling
    np.random.seed(5)
    rows = []
    print('Initiating rows')
    for i in range(m):
        rows += [i]*r
    rows = np.array(rows).reshape((m*r,))
    cols = np.zeros((r*m,))
    print('Initiating cols')    
    for i in range(0, m*r, r):
        cols[i:i+r] = np.random.choice(n, r, replace=False)
    print('Initiating data')
    data = np.ones((m*r,))
    data[1:-1:2] = -1
    print('Initiating matrix')
    Z = sp.csc_matrix((data, (rows, cols)))
    print('Done, returning Z')
    return Z

In [ ]:
m, n, r = 100, 20, 6
Z = generate_data(m, n, r)

## Store/Load Latest generated Z

In [ ]:
# Save
np.save('./data/zdata', sp.coo_matrix(Z).data)
np.save('./data/zrows', sp.coo_matrix(Z).row)
np.save('./data/zcols', sp.coo_matrix(Z).col)

In [2]:
# Load
tmp_data = np.load('./data/zdata.npy')
tmp_rows = np.load('./data/zrows.npy')
tmp_cols = np.load('./data/zcols.npy')
Z = sp.csc_matrix((tmp_data, (tmp_rows, tmp_cols)))
m, n = Z.shape
r = int(len(Z.data)/n)

# Measures
Measures, including number of true predictions, objective function, etc.

In [ ]:
def g0(Z, α):
    print('Number of erronous predictions: %i/%i' % (sum(Z*α <= 0), Z.shape[0]))

# Finding a solver
This section attempts to solve the problem. First is the BFGS, which need an initial guess.

In [3]:
k = 0.01
search_opts = dict(disp=True, maxiter=1000, )  # gtol, norm, return_all, eps not set
def obj(a):
    global Z, k
    p = Z*a  # sparse(m x n) x (n x 1) 
    return np.sum(np.exp(-k*p))

def der_obj(a):
    global Z, k
    p = Z*a  # (m x 1)
    return ((-k) * np.exp(-k*p).transpose() * Z).transpose()  # (1 x m) x sparse(m x n)

In [4]:
a0 = np.zeros((n,))

In [5]:
sol = sopt.minimize(obj, a0, method='BFGS', options=search_opts)
print(sol['jac'])

MemoryError: 

### Jacobian
Attempting to add the Jacobian (gradient) instead of letting BFGS estimate it. The only reason for not doing this is that it is likely that estimating the Jacobian is faster (however it may require more iterations).

In [ ]:
sol = sopt.minimize(obj, a0, jac=der_obj, method='BFGS', options=search_opts)
print(sol['jac'])

In [ ]:
sol['x']

Using the analytical Jacobian seems to help, however it could be situation depend. There are more variables to investigate:
- Increase n, m and r
- Adjust k if necessary
- Generate Z based on true data

Note also that the magnitude of the values of the solution is quite high. When using the model, it is only the relative magnitude that is significant. A future test is to include a penalty on the magnitude of the solution (in which case BFGS can still be used), or impose a constraint. Another idea is to find a function that satisfy 
$f(\alpha) = f(c\alpha),$
for arbitrary $c$ (perhaps $c>0$).

## Increasing the size of Z

Generate the data:

In [ ]:
m, n, r = 10000, 1000, 60
Z = generate_data(m, n, r)

Generate initial guess, then solve

In [ ]:
a0 = np.zeros((n,))

In [ ]:
sol = sopt.minimize(obj, a0, jac=der_obj, method='BFGS', options=search_opts)
print('Norm of gradient: %e' % np.linalg.norm(sol['jac']))

In [ ]:
print(min(sol['x']), max(sol['x']))

The values are close, but it is difficult to conclude that they are close enough. Before moving on to using true data. It is important to address the computational effort required.

# Computation Time
The $m, n, r = (10000, 1000, 60)$ example above required about a minute and 142 iterations to complete. The norm of the gradient was 10e-5, and it terminated successfully. An example with only first order terms would be (100000, 125, 10), and second-order terms $(100000, 125 + 125\cdot126, 10 + 5\cdot5 + 5\cdot 6/2)$:

In [ ]:
(100000, 125 + 125*126, int(10 + 5*5 + 5*(6/2)))

### Investigating true complexity
I will start with the already tested 10000, 1000, 60, and work my way up from there. The genertion process is likely to take significant  time.

In [ ]:
m, n, r = 100000, 15000, 60
Z = generate_data(m,n,r)

The "Store/Load"-section should be used after the data has been generated the first time, instead of optimizing the generation process.

In [ ]:
sol = sopt.minimize(obj, a0, jac=der_obj, method='BFGS', options=search_opts)
print('Norm of gradient: %e' % np.linalg.norm(sol['jac']))

# Spare

In [ ]:
nwins = np.ones((1,m))*((Z+np.abs(Z))/2)
nrounds = np.ones((1,m))*np.abs(Z)
ratio = nwins/nrounds